In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
def rgb_to_color_name(rgb):
    color_name = None
    min_colors = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - rgb[0]) ** 2
        gd = (g_c - rgb[1]) ** 2
        bd = (b_c - rgb[2]) ** 2
        min_colors[(rd + gd + bd)] = name
    if min_colors:
        color_name = min_colors[min(min_colors.keys())]
    return color_name

In [3]:
def solution(model , address):

    global test_arr
    
    data_dict = dict()
    path = address
    model = model
    img_index = 0

    file_names = os.listdir(path)

    for file_name in file_names:

        sub_arr = []
        
        if file_name.endswith(".jpg"):
            
            input_image = Image.open(os.path.join(path, file_name))
            output = model(input_image)
            test_detections = output.pandas().xyxy[0]
            list_test_detections = list(test_detections['name'])
            idx = 0

            for index, row in test_detections.iterrows():
                box = row[['xmin', 'ymin', 'xmax', 'ymax']].values.astype(int)
                cropped_img = np.array(input_image.crop(box))
                test_arr.append(input_image.crop(box))

                # 바운딩 박스 안의 픽셀 색상 평균 계산
                color = np.mean(cropped_img, axis=(0,1)).astype(int)                                  
                    
                sub_arr.append(
                    [list_test_detections[idx] , rgb_to_color_name(tuple(color))]
                )

                idx += 1
            data_dict[img_index] = sub_arr

            img_index += 1
        
    return data_dict

In [4]:
import torch
import torchvision.transforms as transforms
from PIL import Image , ImageDraw
import numpy as np
import matplotlib.pyplot as plt
import webcolors
import os
import urllib.request

In [5]:
!pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.5/586.5 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 25.9 MB/s eta 0:00:00


In [6]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/drive/MyDrive/dmz_teamproject/yolov5/runs/train/exp14/weights/best.pt')

/usr/local/lib/python3.10/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-5-18 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)



requirements: /usr/local/lib/python3.10/dist-packages/requirements.txt not found, check failed.


Fusing layers... 
Model summary: 157 layers, 7031701 parameters, 0 gradients
Adding AutoShape... 


In [7]:
address = '/content/drive/MyDrive/dmz_teamproject/test_dataset/out_data/picture/'

In [8]:
test_arr = []
color = solution(model , address)

In [19]:
color

{0: [['dress', 'darkgray'], ['pants', 'gainsboro']],
 1: [['dress', 'slategray'], ['pants', 'darkgray'], ['pants', 'darkgray']],
 2: [['pants', 'darkkhaki'], ['pants', 'gray']],
 3: [['pants', 'dimgray'], ['shirt', 'lightslategray']],
 4: [['shirt', 'darkslateblue']],
 5: [['pants', 'darkgray'], ['polar', 'darkslategray']],
 6: [['shirt', 'dimgray'], ['dress', 'dimgray'], ['pants', 'darkslategray']],
 7: [['jacket', 'rosybrown']],
 8: [['pants', 'darkslategray']],
 9: [['shirt', 'lightgray'], ['pants', 'rosybrown']]}